In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

import numpy as np

In [2]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

In [3]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

In [4]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30

In [5]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/dataset/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/dataset/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 0,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:18<00:00, 524124.93it/s]


Extracting /dataset/MNIST_models/MNIST\raw\train-images-idx3-ubyte.gz to /dataset/MNIST_models/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 157237.20it/s]


Extracting /dataset/MNIST_models/MNIST\raw\train-labels-idx1-ubyte.gz to /dataset/MNIST_models/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:05<00:00, 322180.56it/s]


Extracting /dataset/MNIST_models/MNIST\raw\t10k-images-idx3-ubyte.gz to /dataset/MNIST_models/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2260117.31it/s]

Extracting /dataset/MNIST_models/MNIST\raw\t10k-labels-idx1-ubyte.gz to /dataset/MNIST_models/MNIST\raw



In [7]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        model.eval()
        
        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(test) : {}".format(test_acc))
        
        model.train()

number of parameters : 2762
Epoch 1, Loss(train) : 438.65160274505615
Epoch 2, Loss(train) : 363.2270836830139
Acc(test) : 0.953125
Epoch 3, Loss(train) : 355.8481649160385
Epoch 4, Loss(train) : 353.26621901988983
Acc(test) : 0.97265625
Epoch 5, Loss(train) : 351.903426527977
Epoch 6, Loss(train) : 350.8769657611847
Acc(test) : 0.97265625
Epoch 7, Loss(train) : 350.25213158130646
Epoch 8, Loss(train) : 349.78890466690063
Acc(test) : 0.9765625
Epoch 9, Loss(train) : 349.39424526691437
Epoch 10, Loss(train) : 349.0026273727417
Acc(test) : 0.984375
Epoch 11, Loss(train) : 348.78305435180664
Epoch 12, Loss(train) : 348.48229134082794
Acc(test) : 0.97265625
Epoch 13, Loss(train) : 348.25154650211334
Epoch 14, Loss(train) : 348.0537129640579
Acc(test) : 0.96484375
Epoch 15, Loss(train) : 347.94418835639954
Epoch 16, Loss(train) : 347.7226630449295
Acc(test) : 0.984375
Epoch 17, Loss(train) : 347.5749111175537
Epoch 18, Loss(train) : 347.4238156080246
Acc(test) : 0.97265625
Epoch 19, Loss(tr

In [10]:
y_.shape

torch.Size([256, 10])

In [11]:
y.shape

torch.Size([256])

In [13]:
y_[0]

tensor([8.0256e-16, 1.1691e-16, 1.5127e-13, 1.3978e-23, 8.9719e-14, 7.2140e-18,
        1.0000e+00, 9.1534e-25, 1.2412e-16, 4.5113e-33], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [14]:
model

MNISTCNN(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2))
  (BN1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(2, 2))
  (BN2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc): Linear(in_features=32, out_features=10, bias=True)
)

In [15]:
x.shape

torch.Size([256, 1, 28, 28])

In [17]:
x[0][0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1608, 0.8510, 0.6824, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2392, 0.9961, 0.7922, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.4549, 0.9961, 0.7922, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [3]:
import torch

In [11]:
def CELoss(output, action, advantage):
    return torch.mean((-1.0)*torch.log(torch.exp(output[action]) / torch.sum(torch.exp(output)))*advantage)

In [16]:
output = torch.tensor([[0.5, 0.2, 0.3], [0.1, 0.4, 0.5], [0.3, 0.3, 0.4]])
action = torch.tensor([0, 1, 2])
advantage = torch.tensor([-1, 0.2, 0.7])

In [17]:
result = CELoss(output, action, advantage)
result

tensor(-0.0979)

In [8]:
output = torch.tensor([[0.5, 0.2, 0.3], [1, -1, 1], [0.3, 0.3, 0.4]])
action = torch.tensor([0, 1, 2])
advantage = torch.tensor([-1, 0.2, 0.7])

In [9]:
result = CELoss(output, action, advantage)
result

tensor(-0.0863)

In [10]:
result / 0.3

tensor(-0.2877)

In [18]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([1, 2, 3])
c = a * b
c

tensor([1, 4, 9])

In [19]:
def CELoss(output, action, advantage):
    return torch.mean((-1.0)*torch.log(torch.exp(output[action]) / torch.sum(torch.exp(output)))*advantage)

In [21]:
output = torch.tensor([[0.18, 0.04, 0.09], [0.51, 0.25, 0.26], [0.51, -0.14, -1.33], [-0.06, 0.33, 0.52]])
action = torch.tensor([2, 0, 1, 0])
advantage = torch.tensor([0.35, 0.28, 0.15, 0.43])

In [24]:
output[action]

tensor([[ 0.5100, -0.1400, -1.3300],
        [ 0.1800,  0.0400,  0.0900],
        [ 0.5100,  0.2500,  0.2600],
        [ 0.1800,  0.0400,  0.0900]])

In [31]:
def CELoss(output, action, advantage):
    size = len(output)
    result = torch.zeros(size)
    for i in range(size):
        value = (-1.0)*torch.log(torch.exp(output[i][action[i]]) / torch.sum(torch.exp(output[i])))*advantage[i]
        result[i] = value
    return torch.mean(result)

In [32]:
print(CELoss(output, action, advantage))

tensor(0.3627)
